# IBD data pathway score correlation network

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from process_met_data import ProcData
from process_pathways import ProcessPathways
import methods
from simulate_met_data import SimulateDataset, SimulateDatasetNamed
import scipy.stats as stats
import statsmodels.api as sm
import helper_functs
import networkx as nx
from sklearn.preprocessing import StandardScaler

## Pathway overlap/score correlation network

In [ ]:
# Load Reactome pathway dictionary
R76 = ProcessPathways("R76", "ChEBI2Reactome_All_Levels.txt", "Homo sapiens")
# R76 = ProcessPathways("R76", "Ensembl2Reactome.txt", "Homo sapiens")
pathway_dict, pathway_names = R76.process_reactome()

In [ ]:
cluster1 = ['R-HSA-442660', 'R-HSA-1614635', 'R-HSA-352230', 'R-HSA-1614558', 'R-HSA-8963693', 'R-HSA-8957322', 'R-HSA-159418', 'R-HSA-192105', 'R-HSA-194068', 'R-HSA-193368', 'R-HSA-1660662', 'R-HSA-1369062', 'R-HSA-382556', 'R-HSA-9018678', 'R-HSA-9025106', 'R-HSA-9018683', 'R-HSA-351143', 'R-HSA-351202', 'R-HSA-2029480', 'R-HSA-2029485', 'R-HSA-211935', 'R-HSA-2162123', 'R-HSA-2142753']


In [ ]:
cluster1 = cluster1.split("\n")

In [ ]:
cluster1 = [i for i in cluster1 if i != ""]

In [ ]:
cluster1

In [ ]:
cluster2 =['R-HSA-70635', 'R-HSA-1222556', 'R-HSA-9006934', 'R-HSA-194138', 'R-HSA-392154', 'R-HSA-418346', 'R-HSA-4420097', 'R-HSA-5218920', 'R-HSA-73894', 'R-HSA-73929', 'R-HSA-73884', 'R-HSA-425397', 'R-HSA-15869', 'R-HSA-8956319', 'R-HSA-74259', 'R-HSA-83936', 'R-HSA-8956321', 'R-HSA-3296197', 'R-HSA-427601', 'R-HSA-196849', 'R-HSA-196854', 'R-HSA-428643', 'R-HSA-196807', 'R-HSA-197264', 'R-HSA-390696', 'R-HSA-5652084', 'R-HSA-71387']


In [ ]:
cluster2 = cluster2.split("\n")
cluster2 = [i for i in cluster2 if i != ""]

In [ ]:
# calculate overlaps
c1_paths = {k: v for k, v in pathway_dict.items() if k in cluster1}

In [ ]:
# calculate overlaps
c2_paths = {k: v for k, v in pathway_dict.items() if k in cluster2}

In [ ]:
import itertools
# path_coverage = {k: [i for i in v if i in compounds_present] for k, v in pathways_present.items()}

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def overlap_coefficient(list1, list2):
    # Szymkiewicz–Simpson coefficient
    intersection = len(list(set(list1).intersection(list(set(list2)))))
    smaller_set = min(len(list1), len(list2))
    return float(intersection) / smaller_set

# all_pathways = [k for k, v in path_coverage.items()]
# jaccard_similarity_list = []
# for pathway_pair in itertools.permutations(all_pathways,2):
#     jaccard_similarity_list.append(jaccard_similarity(data[pathway_pair[0]], data[pathway_pair[1]]))

all_pathways = cluster1
rows = []

for i in all_pathways:
    curr_row = []
    for p in all_pathways:
        curr_row.append(jaccard_similarity(pathway_dict[i], pathway_dict[p]))
    rows.append(curr_row)
    
r_ar = np.array(rows)
j_df = pd.DataFrame(r_ar, index=all_pathways, columns=all_pathways)
j_df.head()

In [ ]:
# calculate correlation between pathway scores

In [ ]:
ibd_data = ProcData("IBD")
ibd_data.process_IBD(id_type="CHEBI")
ibd_data_orig = ibd_data.data_proc

In [ ]:
# Load Reactome pathway dictionary
R76 = ProcessPathways("R76", "ChEBI2Reactome_All_Levels.txt", "Homo sapiens")
pathway_dict, pathway_names = R76.process_reactome()

# Remove large and uninformative pathways
# TODO Remove large and uninformative pathways
remove_paths = ["R-HSA-1430728", "R-HSA-1643685", "R-HSA-382551"]
pathway_dict = {k: v for k, v in pathway_dict.items() if k not in remove_paths}

# Remove pathways not present in the dataset
compounds_present = ibd_data_orig.columns.tolist()
pathways_present = {k: v for k, v in pathway_dict.items() if len([i for i in compounds_present if i in v]) > 1}
print(len(pathways_present))
print(len(compounds_present))

In [ ]:
ibd_data_orig

In [ ]:
from sklearn.decomposition import KernelPCA
def kpca_res(mat, pathways):
    pathway_matrices = []
    pathway_ids = []
    for pathway, compounds in pathways.items():
        single_pathway_matrix = mat.drop(mat.columns.difference(compounds), axis=1)
        if single_pathway_matrix.shape[1] >= 1:
            pathway_matrices.append(single_pathway_matrix.values)
            pathway_ids.append(pathway)

    scores = []
    for n, m in enumerate(pathway_matrices):
        kpca = KernelPCA(n_components=2, kernel="rbf")
        new_data = kpca.fit_transform(m)
        scores.append(new_data[:, 0])
    scores_df = pd.DataFrame(scores, columns=mat.index, index=pathways.keys())
    return scores_df

In [ ]:
scores_df = kpca_res(ibd_data_orig.iloc[:, :-2], pathways_present).T

In [ ]:
corr_mat = scores_df.corr(method="spearman")

In [ ]:
corr_mat

In [ ]:
np.fill_diagonal(corr_mat.values, 0)

In [ ]:
np.fill_diagonal(j_df.values, 0)

In [ ]:
G = nx.from_pandas_adjacency(corr_mat)

In [ ]:
len(nx.nodes(G))

In [ ]:
edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())

In [ ]:
# for all pathways
size_dict = dict(zip(corr_mat.columns, [len(pathway_dict[i]) for i in corr_mat.columns]))
nx.set_node_attributes(G, size_dict, "pathway_size")

# add pathway name as a node attribute
name_dict = dict(zip(corr_mat.columns, [pathway_names[i] for i in corr_mat.columns]))
nx.set_node_attributes(G, name_dict, "pathway_name")

# add cluster participation as a node attribute
participation_dict = dict(zip(corr_mat.columns, [1 if i in cluster1 else 2 if i in cluster2  else 0 for i in corr_mat.columns]))
nx.set_node_attributes(G, participation_dict, "in_cluster")

In [ ]:
# standardise pathway scores
scores_df.iloc[:, :] = StandardScaler().fit_transform(scores_df.iloc[:, :].to_numpy())

In [ ]:
# calculate fold changes 
scores_df["Group"] = ibd_data_orig["Group"]

In [ ]:
ibd_group_t = scores_df[scores_df["Group"] == "IBD"].iloc[:, :-1]
ctrl_group_t = scores_df[scores_df["Group"] != "IBD"].iloc[:, :-1]
fold_changes_t = np.mean(ibd_group_t, axis=0) - np.mean(ctrl_group_t, axis=0)

In [ ]:
fold_changes_t

In [ ]:
# add mean fold change as a node attribute
fc_dict = dict(zip(fold_changes_t.index, fold_changes_t))
nx.set_node_attributes(G, fc_dict, "mean_FC")

In [ ]:
# add pathway coverage as a node attribute
compounds_present = ibd_data_orig.iloc[:, :-2].columns.tolist()
pathways_present = {k: v for k, v in pathway_dict.items() if any(x in compounds_present for x in v)}
path_coverage = {k: list(set(compounds_present) & set(v)) for k, v in pathways_present.items()}
coverage_dict = dict(zip(path_coverage.keys(), [len(i) for i in path_coverage.values()]))
nx.set_node_attributes(G, coverage_dict, "coverage")

In [ ]:
# add average score value based on subtype
scores_df["diagnosis"] = ibd_data_orig["IBD_status"]
cd_avg = np.mean(scores_df[scores_df["diagnosis"] == "CD"].iloc[:, :-1], axis=0)
uc_avg = np.mean(scores_df[scores_df["diagnosis"] == "UC"].iloc[:, :-1], axis=0)
ctrl_avg = np.mean(scores_df[scores_df["diagnosis"] == "nonIBD"].iloc[:, :-1], axis=0)
nx.set_node_attributes(G, cd_avg.to_dict(), "CD_avg")
nx.set_node_attributes(G, uc_avg.to_dict(), "UC_avg")
nx.set_node_attributes(G, ctrl_avg.to_dict(), "CTRL_avg")


In [ ]:
# add pathway size as a node attribute
size_dict = dict(zip(cluster1, [len(pathway_dict[i]) for i in cluster1]))
nx.set_node_attributes(G, size_dict, "pathway_size")

In [ ]:
# add pathway name as a node attribute
name_dict = dict(zip(cluster1, [pathway_names[i] for i in cluster1]))
nx.set_node_attributes(G, name_dict, "pathway_name")

In [ ]:
G.nodes["R-HSA-375280"]

In [ ]:
nx.get_node_attributes(G, "pathway_name").values()

In [ ]:
nx.write_graphml(G, "IBD_subtype_avg_scores_kPCA_top50.graphml")